Simple python script to scan .CSV list for empty rows and removes those rows. Exports new .CSV file with no empty rows.

In [ ]:
import csv

def delete_rows_with_empty_columns(csv_file):
    rows_to_keep = []
    with open(csv_file, 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            if any(cell.strip() == '' for cell in row):
                continue
            rows_to_keep.append(row)

    with open(csv_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(rows_to_keep)

# Example usage
csv_file = 'v2.1.csv'
delete_rows_with_empty_columns(csv_file)

Test script for calculating timegaps.

In [ ]:
import csv
from collections import defaultdict

# Function to calculate the approximate distance between two items with weight
def calculate_weighted_distance(item1, item2, weight):
    return abs(float(item1) - float(item2)) * weight

# Read data from the CSV file and store it in a dictionary of lists
data = defaultdict(list)
current_list = None  # Variable to keep track of the current list
current_data = []  # Variable to store the current data within a list
with open('test.csv', mode='r') as file:
    reader = csv.reader(file)
    for row in reader:
        if len(row) < 2:
            continue
        item_name = row[0]
        item_data = float(row[1])
        if item_data == 0:
            # Start a new list
            current_list = item_name
            current_data = []  # Initialize an empty list for the new list
            current_data.append((item_name, item_data))
            data[current_list] = current_data
        else:
            # Append data to the current data
            current_data.append((item_name, item_data))
            data[current_list] = current_data  # Update data with the current list

# Initialize a dictionary to store the calculated distances with weights
distances = defaultdict(list)

# Calculate distances between all pairs of occurrences of Item1 and Item2 within each list
for key, item_list in data.items():
    for i in range(len(item_list)):
        for j in range(i + 1, len(item_list)):
            item1_name, item1_data = item_list[i]
            item2_name, item2_data = item_list[j]
            distances_key = "-".join(sorted([item1_name, item2_name]))

            # Calculate the weight based on proximity (you can adjust the weight factor)
            weight = 1 / (j - i + 1)

            distance = calculate_weighted_distance(item1_data, item2_data, weight)
            distances[distances_key].append((distance, i, j))

# Calculate the average weighted distances and print time gaps for each pair of items
threshold = 0.5  # Adjust this value as needed
pruned_distances = {}

for key, value in distances.items():
    if len(value) > 0:
        weighted_distances = [distance for distance, _, _ in value]
        max_weight = max([1 / (j - i + 1) for _, i, j in value])
        average_distance = sum(weighted_distances) / len(weighted_distances)
        
        if max_weight > 0 and average_distance / max_weight >= threshold:
            #print(f"Pair: {key}, Weighted Distance: {average_distance}, Max Weight: {max_weight}")
            pruned_distances[key] = average_distance

# Print the pruned distances
for key, value in pruned_distances.items():
    print(f"{key} : {value}")


In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
from collections import defaultdict
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from tkinter import Tk, filedialog, Button, Text, Scrollbar, Frame
import matplotlib.cm as cm
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster import hierarchy

def process_csv():
    file_path = filedialog.askopenfilename(filetypes=[("CSV files", "*.csv")])
    df = pd.read_csv(file_path, header=None)
    items_in_csv = df.iloc[:, 0].unique()
    calculate_timegap(df)

def calculate_timegap(df):
    data = defaultdict(list)
    current_list = None
    current_data = []
    total_shoppers = 0

    for index1 in range(len(df)):
        item_x = df.iloc[index1, 0]
        value_x = df.iloc[index1, 1]
        status_x = df.iloc[index1, 2]

        if (isinstance(status_x, (int, float)) or str(status_x).isdigit() or status_x == 'Good'):
            if value_x == 0:
                total_shoppers += 1
                current_list = item_x
                current_data = [(item_x, value_x)]
                data[current_list] = current_data
            else:
                current_data.append((item_x, value_x))
                data[current_list] = current_data

        distances = defaultdict(list)

        for key, item_list in data.items():
            for i in range(len(item_list)):
                for j in range(i + 1, len(item_list)):
                    item1_name, item1_data = item_list[i]
                    item2_name, item2_data = item_list[j]
                    distances_key = "-".join(sorted([item1_name, item2_name]))

                    weight = 1 / (j - i + 1)

                    distance = abs(float(item1_data) - float(item2_data)) * weight
                    distances[distances_key].append((distance, i, j))

        threshold = 0.5  # Adjust this value as needed
        pruned_distances = {}

        for key, value in distances.items():
            if len(value) > 0:
                weighted_distances = [distance for distance, _, _ in value]
                max_weight = max([1 / (j - i + 1) for _, i, j in value])
                average_distance = sum(weighted_distances) / len(weighted_distances)

                if max_weight > 0 and average_distance / max_weight >= threshold:
                    pruned_distances[key] = average_distance

        sorted_pruned_distances = dict(sorted(pruned_distances.items()))
        timegap_dict = defaultdict(list)
        i = 0

        for key, value in sorted_pruned_distances.items():
            #item1, item2 = key.split('-')
            #pair = (item1, item2)
            #timegap_dict[pair] = [value]
            print(key, value)
            i+=1
            
            if i == 10:
                break

# Create GUI
root = Tk()
root.title("Graph Visualization")

# Create a frame for the main layout
main_frame = Frame(root)
main_frame.pack(fill='both', expand=True)

# Create a frame for the left column (text frames)
left_frame = Frame(main_frame)
left_frame.pack(side='left', padx=10, pady=10)

# Create text widgets and scrollbars for the left column
item_list_text = Text(left_frame, height=15, width=80)
item_list_text.insert('end', "List of Items:\n")
item_list_text.config(state='disabled')
item_list_text.pack(side='top')
item_list_scrollbar = Scrollbar(left_frame, command=item_list_text.yview)
item_list_text['yscrollcommand'] = item_list_scrollbar.set

timegap_text = Text(left_frame, height=15, width=80)
timegap_text.insert('end', "Normalized Timegaps:\n")
timegap_text.config(state='disabled')
timegap_text.pack(side='top')
timegap_scrollbar = Scrollbar(left_frame, command=timegap_text.yview)
timegap_text['yscrollcommand'] = timegap_scrollbar.set

dataset_info_text = Text(left_frame, height=15, width=80)
dataset_info_text.insert('end', "Dataset Information:\n")
dataset_info_text.config(state='disabled')
dataset_info_text.pack(side='top')
dataset_info_scrollbar = Scrollbar(left_frame, command=dataset_info_text.yview)
dataset_info_text['yscrollcommand'] = dataset_info_scrollbar.set

clustered_data_text = Text(left_frame, height=15, width=80)
clustered_data_text.insert('end', "Clustered Data:\n")
clustered_data_text.config(state='disabled')
clustered_data_text.pack(side='top')
clustered_data_scrollbar = Scrollbar(left_frame, command=clustered_data_text.yview)
clustered_data_text['yscrollcommand'] = clustered_data_scrollbar.set

# Create a button for processing the CSV
process_button = Button(left_frame, text="Process CSV and Visualize Graph", command=process_csv)
plt.close('all')

# Pack all the widgets in the left column
timegap_text.pack(side='top', fill='both', expand=True)
#timegap_scrollbar.pack(side='right', fill='y')
dataset_info_text.pack(side='top', fill='both', expand=True)
#dataset_info_scrollbar.pack(side='right', fill='y')
clustered_data_text.pack(side='top', fill='both', expand=True)
#clustered_data_scrollbar.pack(side='right', fill='y')
process_button.pack(side='top', padx=10, pady=10)

# Create a frame for the right column (plots)
right_frame = Frame(main_frame)
right_frame.pack(side='right', padx=10, pady=10, fill='both', expand=True)

root.mainloop()

: 